1. volume_diff_rate in (15, 50)
2. price_delta > 0.5
3. update_time_min in [0935, 1000)
4. 5_day_price_delta in (-10, 5)
5. day_ma_10 > day_ma_20
6. day_open < day_max_2

In [ ]:
import polars as pl
import os
import toml
import time

from utils.gm import *
from datetime import datetime
from dataclasses import dataclass
from dateutil.relativedelta import relativedelta

pl.Config.set_tbl_cols(100)  # 设置显示的最大列数
pl.Config.set_tbl_rows(100)  # 设置显示的最大行数

sub_list = get_index_list("SHSE.000852")


@dataclass
class KlineRecord:
    code: str
    date: str
    time: str
    open: float
    close: float
    volume: float
    high: float
    low: float


class SecSignal:
    def __init__(self, code: str):
        self.code = code
        self.day_10ma = 0.0
        self.day_20ma = 0.0
        self.day_2max = 0.0
        self.day_2delta = 0.0
        self.day_open = -1.0
        self.last_date = ""
        self.minmap = {}

    def shouldFocus(self, code) -> bool:
        if (
            self.day_10ma <= self.day_20ma
            or self.day_2delta > 5.0
            or self.day_2delta < -10.0
        ):
            return False
        return True

    def check(self, record: KlineRecord) -> bool:
        if self.day_open < 0:
            self.day_open = record.open
        if self.day_open > self.day_2max:
            return False

        if record.time < "09:35:00" or record.time > "10:00:00":
            return False
        if (close - open) * 100 / open - 100 < 0.5:
            return False
        if (
            record.time not in self.minmap
            or self.minmap[record.time]["volume_avg_10"] == 0
        ):
            return False
        vol_rate = record.volume / self.minmap[record.time]["volume_avg_10"]
        if vol_rate <= 10:
            return False

        return True

    def load(self, date: str):
        end_time = datetime.strptime(date, "%Y-%m-%d") - relativedelta(days=1)
        start_time = end_time - relativedelta(months=2)
        end_date = end_time.date().strftime("%Y-%m-%d")
        start_date = start_time.date().strftime("%Y-%m-%d")

        dkline = (
            get_dkline(self.code, start_date, end_date)
            .with_columns(
                pl.col("open").shift(1).alias("pre_open"),
                pl.col("close").rolling_mean(window_size=10).alias("day_10ma"),
                pl.col("close").rolling_mean(window_size=20).alias("day_20ma"),
                pl.col("close").rolling_max(window_size=2).alias("day_2max"),
            )
            .with_columns(
                (
                    (pl.col("close") - pl.col("pre_open")) * 100 / pl.col("pre_open")
                ).alias("day_2delta")
            )
        )
        day_info = dkline.to_dicts()[-1]
        self.day_10ma = day_info["day_10ma"]
        self.day_20ma = day_info["day_20ma"]
        self.day_2max = day_info["day_2max"]
        self.day_2delta = day_info["day_2delta"]
        self.last_date = day_info["date"].strftime("%Y-%m-%d")
        # print(day_info)

        df_mkline = get_mkline(self.code, start_date, end_date)
        # print(df_mkline)
        mkline_list = []
        for key, group in df_mkline.group_by(["time"]):
            group = group.with_columns(
                pl.col("volume").rolling_mean(window_size=10).alias("volume_avg_10"),
            )
            mkline_list.append(group)
        mk_infos = (
            pl.concat(mkline_list)
            .sort("date", "time")
            .filter(pl.col("date").dt.strftime("%Y-%m-%d") == self.last_date)
        ).to_dicts()
        for mk_info in mk_infos:
            self.minmap[mk_info["time"].strftime("%H:%M:%S")] = {
                "volume_avg_10": mk_info["volume_avg_10"]
            }
        # print(self.minmap)


signal = SecSignal("SHSE.600682")
signal.load(datetime.now().date().strftime("%Y-%m-%d"))
# print(signal)

shape: (10_489, 9)
┌─────────────┬──────┬──────┬──────┬───────┬────────┬────────────┬──────────┬────────────┐
│ symbol      ┆ open ┆ high ┆ low  ┆ close ┆ volume ┆ date       ┆ time     ┆ turnover   │
│ ---         ┆ ---  ┆ ---  ┆ ---  ┆ ---   ┆ ---    ┆ ---        ┆ ---      ┆ ---        │
│ str         ┆ f64  ┆ f64  ┆ f64  ┆ f64   ┆ i64    ┆ date       ┆ time     ┆ f64        │
╞═════════════╪══════╪══════╪══════╪═══════╪════════╪════════════╪══════════╪════════════╡
│ SHSE.600682 ┆ 6.07 ┆ 6.12 ┆ 6.06 ┆ 6.11  ┆ 437200 ┆ 2024-10-17 ┆ 09:31:00 ┆ 2.663711e6 │
│ SHSE.600682 ┆ 6.11 ┆ 6.11 ┆ 6.08 ┆ 6.08  ┆ 141600 ┆ 2024-10-17 ┆ 09:32:00 ┆ 863147.0   │
│ SHSE.600682 ┆ 6.08 ┆ 6.1  ┆ 6.08 ┆ 6.08  ┆ 161300 ┆ 2024-10-17 ┆ 09:33:00 ┆ 982091.0   │
│ SHSE.600682 ┆ 6.08 ┆ 6.09 ┆ 6.07 ┆ 6.08  ┆ 59600  ┆ 2024-10-17 ┆ 09:34:00 ┆ 362382.0   │
│ SHSE.600682 ┆ 6.08 ┆ 6.09 ┆ 6.07 ┆ 6.07  ┆ 209300 ┆ 2024-10-17 ┆ 09:35:00 ┆ 1.272513e6 │
│ SHSE.600682 ┆ 6.07 ┆ 6.08 ┆ 6.06 ┆ 6.08  ┆ 135000 ┆ 2024-10-17 ┆ 09:3

In [1]:

from futu import *


quote_ctx = OpenQuoteContext(host="127.0.0.1", port=11111)

class StockQuoteHandler(StockQuoteHandlerBase):
    def on_recv_rsp(self, rsp_str):
        ret_code, data = super().on_recv_rsp(rsp_str)
        if ret_code != RET_OK:
            print(f"Error: {ret_code}")
            return
        # 输出实时行情数据
        print(data)
        # df = pl.DataFrame(data)["code", "last_price", "volume", "data_time"]
        # [code, price, volume, data_time] = df.to_numpy().tolist()[0]
        # for strategy in strs:
        #     strategy.check(code, price, volume, data_time)

# 实例化回调处理类
handler = StockQuoteHandler()

# sub_type = SubType.K_1M
sub_type = SubType.QUOTE
sub_list = ['SH.600682',]
for sec_id in sub_list:
    ret_sub, err_message = quote_ctx.subscribe(sec_id, sub_type)
    if ret_sub == RET_OK:
        # 注册回调函数
        quote_ctx.set_handler(handler)
    else:
        print(f"订阅失败: {err_message}")
        
# 保持连接，以接收实时推送
input("按回车键退出程序...\n")

# 关闭行情客户端
quote_ctx.close()

2024-12-18 02:10:18,808 | 150077 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=13389358
2024-12-18 02:10:18,843 | 150077 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=1
